In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = np.load("C:/Users/Gunhee/test/letsgo/npy2.npy",allow_pickle=True).item()

In [3]:
df.keys()

dict_keys(['XM3 E-TECH 하이브리드', 'XM3', 'QM6', 'SM6', '르노 MASTER'])

In [4]:
xm3_e = np.array(df["XM3 E-TECH 하이브리드"])
xm3 = np.array(df['XM3'])
qm6 = np.array(df['QM6'])
sm6 = np.array(df['SM6'])
master = np.array(df['르노 MASTER'])

In [5]:
import tensorflow as tf
import torch

In [6]:
xm3_t = torch.Tensor(xm3)
xm3_e_t = torch.Tensor(xm3_e)
qm6_t = torch.Tensor(qm6)
sm6_t = torch.Tensor(sm6)
master_t = torch.Tensor(master)

In [ ]:
xm3_t.shape

In [8]:
dim_0 = torch.cat((xm3_t,xm3_e_t,qm6_t,sm6_t,master_t),dim=0)

In [9]:
dim_0.shape

torch.Size([5000, 128, 128, 3])

In [10]:
c = np.array(dim_0)

In [43]:
# 라벨 만들기
number_of_data = 5000
y = np.zeros(number_of_data, dtype=np.int32)
y[1000:2000] = 1
y[2000:3000] = 2
y[3000:4000] = 3
y[4000:5000] = 4

In [12]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from torch import optim

In [44]:
x_train, x_test, y_train, y_test = train_test_split(c,y ,random_state=42,test_size=0.25) # 넘파이
# x_train, x_test, y_train, y_test = train_test_split(dim_0,y ,random_state=42) # 토치

In [ ]:
# # 토치
# x_train = torch.as_tensor(x_train)
# x_test = torch.as_tensor(x_test)
# y_train = torch.as_tensor(y_train)
# y_test = torch.as_tensor(y_test)

# ds_train = TensorDataset(x_train, y_train)
# ds_test = TensorDataset(x_test, y_test)

# loader_train = DataLoader(ds_train, batch_size=64, shuffle=True)
# loader_test = DataLoader(ds_test, batch_size=64, shuffle=False)

In [37]:
x_train.shape,y_train.shape, x_test.shape,y_test.shape

((3750, 128, 128, 3), (3750,), (1250, 128, 128, 3), (1250,))

In [45]:
x_train = x_train.astype(float) / 255.0
x_test = x_test.astype(float) / 255.0

In [ ]:
# ss = StandardScaler()
# x_train_s = ss.fit_transform(x_train)
# x_test_s = ss.fit_transform(x_test)

In [ ]:
# df = np.load("C:/Users/Gunhee/test/letsgo/npy1.npy",allow_pickle=True).item()
# caltech_dir = "C:/Users/@/Desktop/product_image/Product_image_recog/image"
# categories = ['XM3 E-TECH 하이브리드', 'XM3', 'QM6', 'SM6', '르노 MASTER']
# nb_classes = len(categories)

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Dense, Dropout, Input,Activation
from tensorflow.keras.models import Model
from keras import optimizers, initializers, regularizers, metrics ,activations
from tensorflow.keras.utils import to_categorical

In [46]:
# 모델 구조 정의 
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(128,128,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5)),

model.add(Conv2D(64, (3, 3), padding='same')),
model.add(Activation('relu')),
model.add(Dropout(0.5)),
model.add(Conv2D(64, (3, 3))),
model.add(MaxPooling2D(pool_size=(2, 2))),
model.add(Dropout(0.5)),

# 전결합층
model.add(Flatten())    # 벡터형태로 reshape
model.add(Dense(512))   # 출력
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(5))
model.add(Activation('softmax'))
# 모델 구축하기
model.compile(loss='sparse_categorical_crossentropy',   # 최적화 함수 지정
    optimizer='rmsprop',
    metrics=['accuracy'])
# 모델 확인
#print(model.summary())

In [ ]:
history1 = model.fit(x_train, y_train, 
                    batch_size=128, 
                    epochs=20, 
                    validation_data=(x_test, y_test))

Epoch 1/20
30/30 [==============================] - 87s 3s/step - loss: 5.9455 - accuracy: 0.2189 - val_loss: 9.8451 - val_accuracy: 0.1936
Epoch 2/20
30/30 [==============================] - 84s 3s/step - loss: 1.6347 - accuracy: 0.2347 - val_loss: 8.8233 - val_accuracy: 0.1944
Epoch 3/20
30/30 [==============================] - 86s 3s/step - loss: 1.6455 - accuracy: 0.2576 - val_loss: 4.3743 - val_accuracy: 0.1888
Epoch 4/20
30/30 [==============================] - 92s 3s/step - loss: 1.5569 - accuracy: 0.2675 - val_loss: 3.0553 - val_accuracy: 0.2240
Epoch 5/20
30/30 [==============================] - 90s 3s/step - loss: 1.5263 - accuracy: 0.3000 - val_loss: 3.2370 - val_accuracy: 0.2160
Epoch 6/20
30/30 [==============================] - 89s 3s/step - loss: 1.4927 - accuracy: 0.3464 - val_loss: 2.3551 - val_accuracy: 0.2952
Epoch 7/20
30/30 [==============================] - 91s 3s/step - loss: 1.4147 - accuracy: 0.3819 - val_loss: 2.6745 - val_accuracy: 0.2480
Epoch 8/20
30/30 [==

In [42]:
# hdf5_file = "C:/Users/Gunhee/test/letsgo\model/7obj-model.hdf5"
hdf5_file = "C:/Users/Gunhee/test/letsgo\model/8obj-model.hdf5"



In [40]:
# 모델의 오차범위
score = model.evaluate(x_test, y_test)
print('loss=', score[0])        # loss
print('accuracy=', score[1])    # acc

40/40 [==============================] - 5s 121ms/step - loss: 3.5853 - accuracy: 0.4480
loss= 3.5852789878845215
accuracy= 0.4480000138282776


In [1]:
# acc = history1.history['acc']
# val_acc = history1.history['val_acc']
# loss = history1.history['loss']
# val_loss = history1.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs, acc, 'bo', label='Training acc')
# plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.legend()

# plt.figure()

# plt.plot(epochs, loss, 'bo', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()

In [19]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
model = Sequential()
model.add(Conv2D(16,(3,3),activation='relu', input_shape=(128,128,3)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(5,activation='softmax'))

model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

earlyStop = tf.keras.callbacks.EarlyStopping(patience=3)

model.fit( x_train, y_train, epochs=500,
          validation_data=( x_test, y_test),
          callbacks=earlyStop
         )